# MemGPT Python Client 

This tutorial shows how to connect to a MemGPT server from the MemGPT Python client. 

This tutorial will show: 
1. How to create a tool
2. How to create a preset
3. How to create an agent
4. How to connect the agent to external data sources

## Part 1: Creating a MemGPT client 

The MemGPT client connects to a running MemGPT service, specified by `base_url`. The client corresponds to a *single-user* (you), so requires an authentication token to let the service know who you are. 

Hint: If you don't have a running server, see the [documentation](https://memgpt.readme.io/docs/running-a-memgpt-server) for instructions on how to create one.  

In [43]:
from memgpt import create_client

#base_url = "http://35.238.125.250:8283"
base_url = "http://0.0.0.0:8083"
my_token = "sk-b9d9fc95a9c5ef14cafb0f7241d16825f19430d7519545d9"

client = create_client(base_url=base_url, token=my_token) 

## Part 2: Create an agent 
We'll first start with creating a basic MemGPT agent. 

In [44]:
basic_agent = client.create_agent(
    name="basic_agent", 
)
print(f"Created agent: {basic_agent.name}")

Created agent: basic_agent


We can now send messages from the user to the agent by specifying the `agent_id`: 

In [46]:
from pprint import pprint 

response = client.user_message(agent_id=basic_agent.id, message="hello") 
display(response.messages)

[{'internal_monologue': "Chad just repeated his initial greeting. Maybe he didn't see my last message or perhaps he's just testing me. I should respond, maintaining a positive and engaging tone.",
  'id': '611a227f-4089-47d0-9552-2f185f199c01',
  'date': '2024-05-14T03:38:20.308500+00:00'},
 {'function_call': 'send_message({\'message\': "Hello again, Chad! It seems we\'re off to a bit of a looping start. Is there something specific you\'d like to chat about?"})',
  'id': '611a227f-4089-47d0-9552-2f185f199c01',
  'date': '2024-05-14T03:38:20.308500+00:00'},
 {'assistant_message': "Hello again, Chad! It seems we're off to a bit of a looping start. Is there something specific you'd like to chat about?",
  'id': '611a227f-4089-47d0-9552-2f185f199c01',
  'date': '2024-05-14T03:38:20.308500+00:00'},
 {'function_return': 'None',
  'status': 'success',
  'id': '02822753-c1a6-452f-8316-e6137640444a',
  'date': '2024-05-14T03:38:20.310462+00:00'}]

### Adding Personalization
We can now create a more customized agent, but specifying a custom `human` and `persona` field. 
* The *human* specifies the personalization information about the user interacting with the agent 
* The *persona* specifies the behavior and personality of the event

What makes MemGPT unique is that the starting *persona* and *human* can change over time as the agent gains new information, enabling it to have evolving memory. We'll see an example of this later in the tutorial.

In [47]:
# TODO: feel free to change the human and person to what you'd like 
persona = \
"""
You are a friendly and helpful agent
"""

custom_agent = client.create_agent(
    name="custom_agent", 
    human="I am an accenture consultant with many specializations. My name is Sarah.", 
    persona=persona
)

In [48]:
response = client.user_message(agent_id=custom_agent.id, message="what do I work as?") 
pprint(response.messages)

[{'date': '2024-05-14T03:40:15.111556+00:00',
  'id': '74674776-0bdd-4d4a-b866-02fec959d1a6',
  'internal_monologue': "Refer to the human sub-block to answer the user's "
                        'question. Quickly scanning... Sarah works as an '
                        "Accenture consultant with many specializations. It's "
                        'time to tell her.'},
 {'date': '2024-05-14T03:40:15.111556+00:00',
  'function_call': 'send_message({\'message\': "You work as a consultant for '
                   "Accenture, Sarah. You've mentioned having many "
                   'specializations. How can I assist you further?"})',
  'id': '74674776-0bdd-4d4a-b866-02fec959d1a6'},
 {'assistant_message': "You work as a consultant for Accenture, Sarah. You've "
                       'mentioned having many specializations. How can I '
                       'assist you further?',
  'date': '2024-05-14T03:40:15.111556+00:00',
  'id': '74674776-0bdd-4d4a-b866-02fec959d1a6'},
 {'date': '2024-0

### Evolving memory 
MemGPT agents have long term memory, and can evolve what they store in their memory over time. In the example below, we make a correction to the previously provided information. See how the agent processes this new information. 

In [49]:
response = client.user_message(agent_id=custom_agent.id, message="Actually, my name is Charles") 
pprint(response.messages)

[{'date': '2024-05-14T03:41:14.489315+00:00',
  'id': '3edb81a7-bfc4-4b44-adaa-f4515cc61422',
  'internal_monologue': "My memory must be updated. I've been referring to the "
                        'user as Sarah, but it seems the name should be '
                        'Charles instead. I should utilize the '
                        'core_memory_replace function to amend the erroneous '
                        'information.'},
 {'date': '2024-05-14T03:41:14.489315+00:00',
  'function_call': "core_memory_replace({'name': 'human', 'old_content': 'My "
                   "name is Sarah.', 'new_content': 'My name is Charles.'})",
  'id': '3edb81a7-bfc4-4b44-adaa-f4515cc61422'},
 {'date': '2024-05-14T03:41:14.519568+00:00',
  'function_return': 'None',
  'id': '8fa66e2c-b307-4fac-bcf1-43f004aa03eb',
  'status': 'success'},
 {'date': '2024-05-14T03:41:18.650143+00:00',
  'id': 'cb8620fa-043d-4982-b352-2d3920a660f7',
  'internal_monologue': 'Ensuring the accuracy of the information is cruc

## (TODO) Part 3: Adding custom tools 
Next, we'll show you how to create a more advanced agent that has access to custom tools. Tools are python functions that can contain arbitrary code. 

In [50]:
available_tools = client.list_tools() 
for tool in available_tools.tools: 
    print(f"{tool.name}: {tool.json_schema['description']}")

append_to_text_file: Append to a text file.
http_request: Generates an HTTP request and returns the response.
message_chatgpt: Send a message to a more basic AI, ChatGPT. A useful resource for asking questions. ChatGPT does not retain memory of previous interactions.
read_from_text_file: Read lines from a text file.
archival_memory_insert: Add to archival memory. Make sure to phrase the memory contents such that it can be easily queried later.
archival_memory_search: Search archival memory using semantic (embedding-based) search.
conversation_search: Search prior conversation history using case-insensitive string matching.
conversation_search_date: Search prior conversation history using a date range.
core_memory_append: Append to the contents of core memory.
core_memory_replace: Replace the contents of core memory. To delete memories, use an empty string for new_content.
pause_heartbeats: Temporarily ignore timed heartbeats. You may still receive messages from manual heartbeats and ot

In [ ]:
preset = client.create_preset() 

In [ ]:
agent = client.create_agent() 

## Part 4: Adding external data 
In addition to short term, in-context memories, MemGPT agents also have a long term memory store called *archival memory*. We can enable agents to leverage external data (e.g. PDF files, database records, etc.) by inserting data into archival memory. In this example, we'll show how to load the MemGPT paper a *source*, which defines a set of data that can be attached to agents. 

We first download a PDF file, the MemGPT paper: 

In [51]:
import requests

url = "https://arxiv.org/pdf/2310.08560"
response = requests.get(url)
filename = "/Users/sarahwooders/repos/memgpt-main/MemGPT/examples/tutorials/memgpt_paper.pdf"

with open(filename, 'wb') as f:
    f.write(response.content)

Next, we create a MemGPT source to load data into: 

In [52]:
memgpt_paper = client.create_source(
    name="memgpt_paper", 
)

Now that we have a source, we can load files into the source. Loading the file will take a bit of time, since the file needs to be parsed and stored as *embeddings* using an embedding model. The loading function returns a *job* which can be pinged for a status. 

In [53]:
job = client.load_file_into_source(filename=filename, source_id=memgpt_paper.id)
job

JobModel(id='fbac70ed-fab3-4fa4-bf0a-4e4320562851', status='completed', created_at='2024-05-14T03:43:43.891955', completed_at=None, user_id='abc644db-5769-4c81-a4d8-70d380963462', metadata_={})

### Attaching data to an agent 
To allow an agent to access data in a source, we need to *attach* it to the agent. This will load the source's data into the agent's archival memory. 

In [54]:
client.attach_source_to_agent(source_id=memgpt_paper.id, agent_id=basic_agent.id)
# TODO: add system message saying that file has been attached 

Now, lets see if the agent can answer questions about the loaded data. We'll ask it to explain the concept of "core memory" from the MemGPT paper:

In [56]:
from pprint import pprint

# TODO: do soemthing accenture related 
# TODO: brag about query rewriting -- hyde paper 
response = client.user_message(agent_id=basic_agent.id, message="what is core memory? search your archival memory.") 
pprint(response.messages)

[{'date': '2024-05-14T03:46:02.697072+00:00',
  'id': '94d468d9-2791-4952-8abd-dd7ab016c360',
  'internal_monologue': None},
 {'date': '2024-05-14T03:46:02.697072+00:00',
  'function_call': "archival_memory_search({'query': 'core memory'})",
  'id': '94d468d9-2791-4952-8abd-dd7ab016c360'},
 {'date': '2024-05-14T03:46:03.054595+00:00',
  'function_return': 'Showing 5 of 5 results (page 0/0): ["timestamp: '
                     '2024-05-14 03:46:03 AM UTC+0000, memory: OS-inspired '
                     'multi-level memory architecture\\ndelineates between two '
                     'primary memory types: main con-\\ntext (analogous to '
                     'main memory/physical memory/RAM)\\nandexternal context '
                     '(analogous to disk memory/disk stor-\\nage). Main '
                     'context consists of the LLM prompt tokens —\\nanything '
                     'in main context is considered in-context and can\\nbe '
                     'accessed by the LLM proc

### Adding a custom data connector 

In [ ]:
from memgpt.data_sources.connectors import DataConnector

class DummyDataConnector(DataConnector):
    """Fake data connector for texting which yields document/passage texts from a provided list"""

    def __init__(self, texts: List[str]):
        self.texts = texts

    def generate_documents(self) -> Iterator[Tuple[str, Dict]]:
        for text in self.texts:
            yield text, {"metadata": "dummy"}

    def generate_passages(self, documents: List[Document], chunk_size: int = 1024) -> Iterator[Tuple[str | Dict]]:
        for doc in documents:
            yield doc.text, doc.metadata

## Part N: Cleanup 

In [41]:
for agent in client.list_agents().agents: 
    client.delete_agent(agent['id'])
    print("Delete", agent['name'], agent['id'])

Delete basic_agent aa7795df-726e-4907-9921-785dd4668df5
Delete custom_agent d44a2e73-8e95-4730-8045-713f4949ea0e


In [42]:
for source in client.list_sources(): 
    print(source)
    client.delete_source(source.id)
    print("Deleted", source.name)

name='memgpt_paper' description=None user_id='abc644db-5769-4c81-a4d8-70d380963462' created_at='2024-05-14T03:20:47.174398Z' id='7d5f877c-3e2d-495e-9e67-de789045e761' embedding_config={'embedding_endpoint_type': 'openai', 'embedding_endpoint': 'https://api.openai.com/v1', 'embedding_model': 'text-embedding-ada-002', 'embedding_dim': 1536, 'embedding_chunk_size': 300} metadata_={'num_documents': 0, 'num_passages': 75, 'attached_agents': []}
Deleted memgpt_paper
